In [1]:
# design a sinc RF pulse and simulate it
using BlochSim
using Plots
plotly()
using STFR#: getrf # to design an initial pulse we can simulate
using ForwardDiff
using ProgressMeter
using MAT  # to load designed pulse

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots C:\Users\Stellarlet\.julia\packages\Plots\XuV6v\src\backends.jl:372


In [2]:
#Define Brian Hargreave's Bloch simulator for arbitary phased rf pulses
function blochsim_B1_arb(α)
    #α= rfp(abs) + rfp(angle) + b1 + Mx0 + My0 + Mz0
    #rfp = rfp_bs + rfp_ss;
    
    nt = floor(Int,(length(α) - 4) / 2); # might add error msg for wrong length later
    # rfp_abs = α[1 : nt];
    # rfp_angle = α[nt + 1 : nt * 2];
    
    mx = α[end-2];
    my = α[end-1];
    mz = α[end];
    
    for tt = 1:nt
        rf_b1 = α[tt] * α[end - 3];
        
        ca = cos(α[nt+tt]);
        sa = sin(α[nt+tt]);
        
        cb = cos(rf_b1);
        sb = sin(rf_b1);
        
        mx_new = (ca * ca + sa * sa * cb) * mx + (sa * ca - sa * ca * cb) * my + (sa * sb) * mz;
        my_new = (sa * ca - sa * ca * cb) * mx + (sa * sa + ca * ca * cb) * my + (- ca * sb) * mz;
        mz_new = (- sa * sb) * mx + (ca * sb) * my + (cb) * mz;
        
        mx = mx_new;
        my = my_new;
        mz = mz_new;
        
    end

    return [mx my mz]
end

blochsim_B1_arb (generic function with 1 method)

In [19]:
# test the previous BlochSim function
vars = matread("test_pulses/bsse_pulse_raw.mat")

#b1 = (0:0.004:0.004*1023)
b1 = 0:0.025:3;  # Gauss
nb1 = length(b1);

Mxd = zeros(nb1, 1)
Myd = zeros(nb1, 1)
Mzd = zeros(nb1, 1)
Mx0 = 0
My0 = 0
Mz0 = 1.0

rfp = transpose(vars["rfp_bs"] + vars["rfp_ss"]);
dt = 1e-6;
rfp_abs = broadcast(abs, rfp);
rfp_abs = 2 .* pi .* 4258 .* dt .* rfp_abs;
rfp_angle = broadcast(angle, rfp);

@showprogress 1 "Computing..." for ii = 1 : nb1
    rfg = [rfp_abs; rfp_angle; b1[ii]; Mx0; My0; Mz0];
    #rfg = [rfp; b1[ii]; Mx0; My0; Mz0];
    M = blochsim_B1_arb(rfg)
    Mxd[ii] = M[1]
    Myd[ii] = M[2]
    Mzd[ii] = M[3]
end

plot(abs.(Complex.(Mxd, Myd)), label="|Mxy|")
plot!(Mxd, label="Mx")
plot!(Myd, label="My")
plot!(Mzd, label="Mz")

<!DOCTYPE html>
 
 
 Plots.jl

In [20]:
# define a BlochSim we can use to calculate error - should encapsulate myBlochSim here
#also slightly modified - HS 6/10
function myBlochSimErr(α)
       
    # α: vector of RF rotations, plus a gradient rotation at the end, plus a target vector, plus an error weight
        
    Mxd = α[end-3]
    Myd = α[end-2]
    Mzd = α[end-1]     
    w = α[end]
    
    Mx, My, Mz = blochsim_B1_arb(α[1 : end - 4])
    
    err = w * ((Mx - Mxd)^2 + (My - Myd)^2 + (Mz - Mzd)^2)
    # err = w * (Mz - Mzd)^2
    return err
    
end

myBlochSimErr (generic function with 1 method)

In [5]:
println("Start forward diff")
g = b1 -> ForwardDiff.gradient(myBlochSimErr, b1) # optimization gradient
println(g)

Start forward diff
#1


In [21]:
# sanity check to optimize the input with its original output
step = 0.0001
niters = 5
Mx0 = 0
My0 = 0
Mz0 = 1.0
rf_op = [rfp_abs;rfp_angle]
N = length(rf_op)

println("Start iterations")
@showprogress 1 "Computing..." for nn = 1 : niters
    J = zeros(N, 1)
    for ii = 1 : nb1
        rfg = [rf_op; b1[ii]; Mx0; My0; Mz0; Mxd[ii]; Myd[ii]; Mzd[ii]; 1]
        J += g(rfg)[1 : end - 8] 
    end
    rf_op -= step * J
end
plot(rf_op)

Start iterations

Computing... 40%|████████████████                       |  ETA: 0:02:25

Computing...100%|███████████████████████████████████████| Time: 0:03:52


<!DOCTYPE html>
 
 
 Plots.jl

In [26]:
# pulse design for the excitation pulse

# set up target pattern

# b1 = 0:0.025:3;  # Gauss
# nb1 = length(b1);

Mzd = ones(nb1, 1);
Mzd[abs.(collect(b1) .- 82 ./nb1*3) .< (7 ./nb1*3)] .= 0;
w = ones(nb1, 1);
w[abs.(collect(b1) .- 75 ./nb1*3) .< (2 ./nb1*3)] .= 0;
w[abs.(collect(b1) .- 89 ./nb1*3) .< (2 ./nb1*3)] .= 0;

plot(Mxd)
plot!(Myd)
plot!(Mzd)
plot!(w)

<!DOCTYPE html>
 
 
 Plots.jl

In [15]:
step = 0.0001
niters = 15
Mx0 = 0
My0 = 0
Mz0 = 1.0
rf_op = [rfp_abs;rfp_angle]
N = length(rf_op)

println("Start iterations")
@showprogress 1 "Computing..." for nn = 1 : niters
    J = zeros(N, 1)
    for ii = 1 : nb1
        rfg = [rf_op; b1[ii]; Mx0; My0; Mz0; Mxd[ii]; Myd[ii]; Mzd[ii]; w[ii]]
        J += g(rfg)[1 : end - 8] 
    end
    rf_op -= step * J
end
plot(rf_op)

Start iterations


Computing...100%|███████████████████████████████████████| Time: 0:11:21


<!DOCTYPE html>
 
 
 Plots.jl

In [22]:
# Simulate the results -- idk if it looks great but let's see...
Mx = zeros(nb1, 1)
My = zeros(nb1, 1)
Mz = zeros(nb1, 1)
Mx0 = 0
My0 = 0
Mz0 = 1.0
for ii = 1 : nb1
    rfg = [rf_op; b1[ii]; Mx0; My0; Mz0];
    M = blochsim_B1_arb(rfg)
    Mx[ii] = M[1]
    My[ii] = M[2]
    Mz[ii] = M[3]
end

display(plot(g(b1)))
plot(abs.(Complex.(Mx, My)), label="|Mxy|")
plot!(Mx, label="Mx")
plot!(My, label="My")
plot!(Mz, label="Mz")

<!DOCTYPE html>
 
 
 Plots.jl

<!DOCTYPE html>
 
 
 Plots.jl

In [23]:
plot(abs.(Complex.(Mx, My))-abs.(Complex.(Mxd, Myd)), label="|Mxy|")
plot!(Mx-Mxd, label="Mx")
plot!(My-Myd, label="My")
plot!(Mz-Mzd, label="Mz")

<!DOCTYPE html>
 
 
 Plots.jl

In [24]:
plot([rfp_abs;rfp_angle])

<!DOCTYPE html>
 
 
 Plots.jl

In [25]:
plot(rf_op-[rfp_abs;rfp_angle])

<!DOCTYPE html>
 
 
 Plots.jl